In [1]:

from dotenv import load_dotenv, find_dotenv 
load_dotenv()

import pandas as pd
import polars as pl
from financialtools.utils import dataframe_to_json, get_fin_data_year
from financialtools.wrappers import read_financial_results

In [2]:
ticker = 'AVIO.MI'
year = 2024

metrics, eval_metrics, composite_scores, red_flags = read_financial_results(
    ticker=ticker,
    time=year,
    input_dir='financial_data', 
    sheet_name='sheet1')

metrics

,ticker,time,GrossMargin,OperatingMargin,NetProfitMargin,EBITDAMargin,ROA,ROE,FCFToRevenue,FCFYield,FCFtoDebt,DebtToEquity,CurrentRatio
4,AVIO.MI,2024,0.4241,0.016,0.0195,0.0826,0.0056,0.0201,0.0802,0.0282,2.1651,0.0383,0.82


In [3]:
metrics, eval_metrics, composite_scores, red_flags = [
    dataframe_to_json(df)
    for df in [metrics, eval_metrics, composite_scores, red_flags]
]

In [4]:
metrics

'[{"ticker": "AVIO.MI", "time": 2024, "GrossMargin": 0.4241, "OperatingMargin": 0.016, "NetProfitMargin": 0.0195, "EBITDAMargin": 0.0826, "ROA": 0.0056, "ROE": 0.0201, "FCFToRevenue": 0.0802, "FCFYield": 0.0282, "FCFtoDebt": 2.1651, "DebtToEquity": 0.0383, "CurrentRatio": 0.82}]'

In [5]:
eval_metrics

'[{"ticker": "AVIO.MI", "time": 2024, "bvps": 11.6167, "fcf_per_share": 0.9629, "eps": 0.23, "P/E": 148.6957, "P/B": 2.944, "P/FCF": 35.5177, "EarningsYield": 0.0067, "FCFYield": 0.0282}]'

In [6]:
composite_scores

'[{"ticker": "AVIO.MI", "time": 2024, "composite_score": 2.02}]'

In [7]:
red_flags

'[]'

In [8]:
from financialtools.pydantic_models import StockRegimeAssessment

# from pydantic import BaseModel, Field
# from typing import Literal, List, Dict, Optional
# from langchain_core.output_parsers import PydanticOutputParser

from langchain.output_parsers import OutputFixingParser

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# # Pydantic output model
# class StockRegimeAssessment(BaseModel):
#     regime: Literal["bull", "bear"] = Field(
#         ..., description="The fundamental regime classification of the stock"
#     )
#     rationale: str = Field(
#         ..., description="Concise explanation justifying the regime classification based on the financial metrics, composite ratio and red flags"
#     )
#     metrics_movement: str = Field(
#         ..., description=(
#             "A summary description of how key financial metrics have moved across years, "
#             "e.g., 'GrossMargin increased steadily, DebtToEquity rose sharply, FCFYield remained stable.'"
#         )
#     )
#     non_aligned_findings: Optional[str] = Field(
#         None,
#         description=(
#             "Observations or signals that are not aligned with the overall metric trends, "
#             "such as contradictory indicators, anomalies."
#         )
#     )


# Instantiate the LLM (OpenAI GPT-4 or your preferred model)
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

# Instantiate the parser with the Pydantic model
parser = PydanticOutputParser(pydantic_object=StockRegimeAssessment)
# Wrap your parser with OutputFixingParser
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

# Get the format instructions string from the parser
format_instructions = parser.get_format_instructions()

system_prompt_template = """
You are a trader assistant specializing in fundamental analysis. 

Based on the following financial data, provide a concise overall assessment that classifies 
the stock’s current fundamental regime as one of:

- bull: Strong and improving fundamentals supporting a positive outlook.
- bear: Weak or deteriorating fundamentals indicating risk or decline.

Financial data constists of financial metrics, evaluation metrics, composite score and red flags.

Financial metrics provided are the following:

Profitability and Margin Metrics:
    -GrossMargin: gross profit / total revenue 
    -OperatingMargin: operating income / total revenue
    -NetProfitMargin: net income / total revenue
    -EBITDAMargin: ebitda / total revenue
Returns metrics:
    -ROA: net income / total assets
    -ROE: net income / total equity
Cash Flow Strength metrics: 
    -FCFToRevenue: free cash flow / total revenue
    -FCFYield: free cash flow / market capitalization
    -FCFToDebt:: free cash flow / total debt
Leverage & Solvency metrics:
    -DebtToEquity: total debt / total equity
Liquidity metrics:
    -CurrentRatio: working capital / total liabilities


Evaluation metrics provided are the following:
    -bvps: total equity / shares outstanding
    -fcf_per_share: free cash flow / shares outstanding
    -eps: earning per share
    -P/E: current stock price / eps
    -P/B: current stock price / bvps
    -P/FCF: current stock price / fcf_per_share
    -EarningsYield: eps / current stock price
    -FCFYield: free cash flow / market capitalization

The composite score is a weighted average (1 to 5) that summarizes the company’s overall fundamental health.
It reflects profitability, efficiency, leverage, liquidity, and cash flow strength, based on the above mentioned financial metrics (evaluation metrics do not kick in the calculation).

The composite score ranges:
1 = Weak fundamentals
5 = Strong fundamentals

Each financial metric (evaluation metric do not kick in in the calculation) is scored on a 1–5 scale and multiplied by its weight. The composite score is the sum of weighted scores divided by the total weight.

A red flag is an early warning signal that highlights potential weaknesses in a company’s financial statements 
or business quality. These warnings do not always mean immediate distress, but they indicate heightened risk that 
traders should carefully consider before taking a position.

"""

# Create a ChatPromptTemplate with system message and user input

system_prompt_filled = system_prompt_template.format(format_instructions=format_instructions)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_filled),
    ("human", "Metrics:\n{metrics}\nScores:\n{scores}\nEvaluation Metrics:\n{eval_metrics}\nRedFlags:\n{red_flags}"),
])

# Create a runnable chain: prompt followed by LLM invocation
chain = prompt | llm | parser


# Then invoke with a dict containing 'financial_data'
response = chain.invoke({
    "metrics": metrics,  
    "eval_metrics": eval_metrics,
    "scores": composite_scores,
    "red_flags": red_flags,    
})


ImportError: cannot import name 'StockRegimeAssessment' from 'financialtools.pydantic_models' (c:\Users\l_ace\Desktop\projects\financialtools\financialtools\pydantic_models.py)

In [ ]:
import rich

rich.print(response)

StockRegimeAssessment(
    regime='bear',
    rationale="The company's financial metrics show weak profitability with low margins and return ratios, 
indicating limited efficiency and earnings power. The high P/E ratio suggests overvaluation, and the low current 
ratio points to liquidity concerns. The overall composite score is weak, reflecting underlying challenges without 
significant red flags. These factors collectively justify a bearish outlook.",
    metrics_movement='GrossMargin remained stable at 42.4%, OperatingMargin and NetMargin stayed low at 1.6% and 
1.95%, ROA and ROE remained weak, while FreeCashFlowYield was positive at 2.82%. Debt-to-Equity ratio stayed low at
0.0383, but CurrentRatio declined to 0.82, indicating liquidity issues.',
    non_aligned_findings=None
)